# Campus ratings

## A scripted lesson

This notebook takes my [Excel pivot table lesson](https://docs.google.com/document/d/1PRM1ozgbqkq69ZwpRue1ttho-FCHeKKR7Thybz6AAak/edit#heading=h.h6x8isam3qkn) and scripts it using agate. It draws from the original data set, instead of the cut down version used in the Excel lesson.

## About the data

The Texas Education Agency rates public schools based on test scores and other factors. This lesson is based on the 2017 ratings released August 15, 2017.

### To download

Go to [Data download](https://rptsvr1.tea.texas.gov/perfreport/account/2017/download.html).
1. Choose "Campus-level Data".
2. Leave on "Accountability Rating, Index Scores, and Reference"
3. Click "Continue.

On the Campus Data Download page:
1. Choose format "Commas-delimted'
2. Select All fields
3. Click Download

This file has been stored in the `/data/` folder.

Note the [fields definitions](https://rptsvr1.tea.texas.gov/perfreport/account/2017/download/camprate.html), which are used for import.

## Goal

We want to find a number of things from this data:
- What percentage of charter schools received a "Needs Improvement" rating, compared to traditional public schools.
- Which schools in Austin ISD received a "Needs Improvement" rating?
- Which schools in Region 13 received a "Needs Improvement" rating?


In [1]:
import agate
import warnings
warnings.filterwarnings('ignore')

In [2]:
# this list of types was build from the field definitions referenced above
specified_type = {
    'CAMPUS': agate.Text(),
    'CAMPNAME': agate.Text(),
    'CNTYNAME': agate.Text(),
    'COUNTY': agate.Text(),
    'DISTNAME': agate.Text(),
    'DISTRICT': agate.Text(),
    'PAIRNAME': agate.Text(),
    'REGNNAME': agate.Text(),
    'REGION': agate.Text(),
    'C_RATING': agate.Text(),
    'C_UPDATE': agate.Text(),
    'CAD_MATH': agate.Text(),
    'CAD_POST': agate.Text(),
    'CAD_READ': agate.Text(),
    'CAD_SCIE': agate.Text(),
    'CAD_SOCI': agate.Text(),
    'CAD_GAP': agate.Text(),
    'CAD_PROGRESS': agate.Text(),
    'C_YRS_IR': agate.Number(),
    'GRDSPAN': agate.Text(),
    'GRDHIGH': agate.Text(),
    'GRDLOW': agate.Text(),
    'PAIRCAMP': agate.Text(),
    'GRDTYPE': agate.Text(),
    'CFLAEATYPE': agate.Text(),
    'CFLAEC': agate.Text(),
    'CFLPAIRD': agate.Text(),
    'CFLCHART': agate.Text(),
    'CFLDAEP': agate.Text(),
    'CFLEEK': agate.Text(),
    'CFLJJ': agate.Text(),
    'CFLNEWCAMP': agate.Text(),
    'CFLPAIR': agate.Text(),
    'CFLPAIRO': agate.Text(),
    'CFLALTED': agate.Text(),
    'CFLRTF': agate.Text(),
    'CI1_CUT': agate.Number(),
    'CI1_MET': agate.Text(),
    'CI1_MAXPTS': agate.Number(),
    'CI1_TOTPTS': agate.Number(),
    'CI1': agate.Number(),
    'CI2_CUT': agate.Number(),
    'CI2': agate.Number(),
    'CI2_MAXPTS': agate.Number(),
    'CI2_MET': agate.Text(),
    'CI2_TOTPTS': agate.Number(),
    'CI3_CUT': agate.Number(),
    'CI3_MAXPTS': agate.Number(),
    'CI3_MET': agate.Text(),
    'CI3': agate.Number(),
    'CI3_TOTPTS': agate.Number(),
    'CI4_CUT': agate.Number(),
    'CI4': agate.Number(),
    'CI4_MET': agate.Text(),
    'CI4_GRD_WGT': agate.Number(),
    'CI4_PSG_WGT': agate.Number(),
    'CI4_RHS_WGT': agate.Number(),
    'CI4_STR_WGT': agate.Number(),
    'CPETECHC': agate.Number(),
    'CPETECOC': agate.Number(),
    'CPETLEPC': agate.Number(),
    'CPEMALLC': agate.Number(),
    'CPETSPEC': agate.Number(),
    'CPETECHP': agate.Number(),
    'CPETECOP': agate.Number(),
    'CPETLEPP': agate.Number(),
    'CPETSPEP': agate.Number(),
    'CPEMALLT': agate.Number(),
    'CPEMALLP': agate.Number(),
    'CPETALLC': agate.Number(),
}

# import the raw data
raw = agate.Table.from_csv('../data/CAMPRATE_csv.dat', column_types=specified_type)


In [3]:
# print columns
print(raw)

| column       | data_type |
| ------------ | --------- |
| CAMPUS       | Text      |
| CAD_GAP      | Text      |
| CAD_MATH     | Text      |
| CAD_POST     | Text      |
| CAD_PROGRESS | Text      |
| CAD_READ     | Text      |
| CAD_SCIE     | Text      |
| CAD_SOCI     | Text      |
| CAMPNAME     | Text      |
| CFLAEATYPE   | Text      |
| CFLAEC       | Text      |
| CFLALTED     | Text      |
| CFLCHART     | Text      |
| CFLDAEP      | Text      |
| CFLEEK       | Text      |
| CFLJJ        | Text      |
| CFLNEWCAMP   | Text      |
| CFLPAIR      | Text      |
| CFLPAIRD     | Text      |
| CFLPAIRO     | Text      |
| CFLRTF       | Text      |
| CI1          | Number    |
| CI1_CUT      | Number    |
| CI1_MAXPTS   | Number    |
| CI1_MET      | Text      |
| CI1_TOTPTS   | Number    |
| CI2          | Number    |
| CI2_CUT      | Number    |
| CI2_MAXPTS   | Number    |
| CI2_MET      | Text      |
| CI2_TOTPTS   | Number    |
| CI3          | Number    |
| CI3_CUT     

In [4]:
# looking at the first 5 records in the table
raw.limit(5).select('CFLCHART').print_table()

| CFLCHART |
| -------- |
| N        |
| N        |
| N        |
| N        |
| N        |


## Creating a column for charter status

In this data set, we have a column that designates if a school is a charter school, using "N" or "Y". We'll use it to create a new column with nicer names.

In [5]:
# this is a function for the .compute method below
# it evaluates if the value sent it is 'Y', indicating it is  a charter
# school. It returns 'Charter'. If not, then 'Not charter'.
def set_charter_column(value):
    if value == 'Y':
        return 'Charter'
    else:
        return 'Not charter'

# We are creating a new column called 'CHARTER'. We take the value in CFLCHART
# and pass it through the function to decide if we print "Charter" or "Not charter"
# We put this all into a new table 
charter_set = raw.compute([
  ('CHARTER', # the name of the new column
   agate.Formula(agate.Text(),
   lambda r: set_charter_column(r['CFLCHART']))
  )
])

In [6]:
# peek at charter records
charter_set.where(lambda row: row['CFLCHART'] == 'Y').select([
        'CAMPNAME',
        'CHARTER'
    ]).limit(5).print_table(max_column_width=None)

| CAMPNAME                           | CHARTER |
| ---------------------------------- | ------- |
| PINEYWOODS COMMUNITY ACADEMY H S   | Charter |
| DR TERRY ROBBINS MIDDLE            | Charter |
| SARAH STRINDEN EL                  | Charter |
| ST MARY'S ACADEMY CHARTER SCHOOL   | Charter |
| RICHARD MILBURN ALTER H S (KILLEEN | Charter |


In [7]:
# peek at non-charter records
charter_set.where(lambda row: row['CFLCHART'] == 'N').select([
        'CAMPNAME',
        'CHARTER'
    ]).limit(5).print_table(max_column_width=None)

| CAMPNAME       | CHARTER     |
| -------------- | ----------- |
| CAYUGA H S     | Not charter |
| CAYUGA MIDDLE  | Not charter |
| CAYUGA EL      | Not charter |
| ELKHART H S    | Not charter |
| ELKHART MIDDLE | Not charter |


## Create column of explained ratings

We'll do something similar for the ratings. Instead of using "I", we want the word "Improvement required".

In [8]:
# These are the values for the rating.
# C_RATING is on the left, the definition is on the right
# M=Met Standard, A=Met Alternative Standard, I=Improvement Required, X/Z=Not Rated, T=Not Rated: Annexation
rating_values = {
    'I': 'Improvement required',
    'M': 'Met standard',
    'A': 'Met alternative standard',
    'X': 'Not rated',
    'Z': 'Not rated',
    'T': 'Not rated',
    '': 'Not rated',
}

# function to get the match of the rating
def map_rating(rating):
    rating = rating.strip()
    return rating_values[rating]

# create the column, and insert the value from the map above
rating_set = charter_set.compute([
  ('RATING',
   agate.Formula(agate.Text(),
   lambda r: map_rating(r['C_RATING']))
  )
])

# Filter out campuses we don't want

We don't want to consider campuses that use the alternative standard, so let's filter those out.

In [9]:
rating_filtered = rating_set.where(
    lambda row: row['RATING'] in ('Met standard', 'Improvement required')
)

print('number of all campuses: {}'.format(len(rating_set)))
print('number after filtering: {}'.format(len(rating_filtered)))
print('distinct values in Ratings now: {}'.format(
        rating_filtered.columns['RATING'].values_distinct()
    ))

number of all campuses: 8757
number after filtering: 7949
distinct values in Ratings now: ('Improvement required', 'Met standard')


In [10]:
# filter where CFLALTED is not true
alts_removed = rating_filtered.where(
    lambda row: row['CFLALTED'] == 'N'
)
print('Number before filter: {}'.format(len(rating_filtered)))
print('Number after filter: {}'.format(len(alts_removed)))

Number before filter: 7949
Number after filter: 7931


In [11]:
print(alts_removed)

| column       | data_type |
| ------------ | --------- |
| CAMPUS       | Text      |
| CAD_GAP      | Text      |
| CAD_MATH     | Text      |
| CAD_POST     | Text      |
| CAD_PROGRESS | Text      |
| CAD_READ     | Text      |
| CAD_SCIE     | Text      |
| CAD_SOCI     | Text      |
| CAMPNAME     | Text      |
| CFLAEATYPE   | Text      |
| CFLAEC       | Text      |
| CFLALTED     | Text      |
| CFLCHART     | Text      |
| CFLDAEP      | Text      |
| CFLEEK       | Text      |
| CFLJJ        | Text      |
| CFLNEWCAMP   | Text      |
| CFLPAIR      | Text      |
| CFLPAIRD     | Text      |
| CFLPAIRO     | Text      |
| CFLRTF       | Text      |
| CI1          | Number    |
| CI1_CUT      | Number    |
| CI1_MAXPTS   | Number    |
| CI1_MET      | Text      |
| CI1_TOTPTS   | Number    |
| CI2          | Number    |
| CI2_CUT      | Number    |
| CI2_MAXPTS   | Number    |
| CI2_MET      | Text      |
| CI2_TOTPTS   | Number    |
| CI3          | Number    |
| CI3_CUT     

## Analysis time

Now that we have our data in all filtered, we'll set our new table and start working with it.

In [12]:
# our final table for analysis

campus = alts_removed

In [13]:
# pivot the table based charter and rating to see the number of records
campus_pivot = campus.pivot('CHARTER', 'RATING')
campus_pivot.print_table()

| CHARTER     | Met standard | Improvement required |
| ----------- | ------------ | -------------------- |
| Not charter |        7,148 |                  299 |
| Charter     |          443 |                   41 |


In [14]:
# function to create fail rate: part / total * 100
def pass_rate(row):
    return ((row['Improvement required'] / (row['Met standard'] + row['Improvement required'])) *100)

In [15]:
# Create new column with fail rate
campus_charter_rate = campus_pivot.compute([
    ('Fail rate', agate.Formula(agate.Number(), pass_rate))
])

# print the new table
campus_charter_rate.print_table()

| CHARTER     | Met standard | Improvement required | Fail rate |
| ----------- | ------------ | -------------------- | --------- |
| Not charter |        7,148 |                  299 |    4.015… |
| Charter     |          443 |                   41 |    8.471… |


## Next: Filter to see Austin ISD schools that failed

In [16]:
# Get just the austin schools
austin = campus.where(lambda row: row['DISTNAME'] == 'AUSTIN ISD')
print(len(austin))

117


In [17]:
# Show the number of schools
austin_pivot = austin.pivot('CHARTER', 'RATING')
austin_pivot.print_table()

| CHARTER     | Met standard | Improvement required |
| ----------- | ------------ | -------------------- |
| Not charter |          113 |                    4 |


In [18]:
# filter the austin list to failed schools
austin_failed = austin.where(lambda row: row['RATING'] == 'Improvement required')

# columns for fail print list
columns_fail_list = [
    'CAMPNAME',
    'DISTNAME',
    'C_YRS_IR',
    'CI1_MET',
    'CI2_MET',
    'CI3_MET',
    'CI4_MET',
]

# print the list of schools
austin_failed.select(columns_fail_list).print_table(max_columns=None)

| CAMPNAME      | DISTNAME   | C_YRS_IR | CI1_MET | CI2_MET | CI3_MET | CI4_MET |
| ------------- | ---------- | -------- | ------- | ------- | ------- | ------- |
| BURNET M S    | AUSTIN ISD |        2 | N       | N       | N       | Y       |
| MARTIN MIDDLE | AUSTIN ISD |        1 | N       | N       | Y       | Y       |
| MENDEZ M S    | AUSTIN ISD |        4 | N       | N       | N       | N       |
| GOVALLE EL    | AUSTIN ISD |        1 | N       | Y       | N       | Y       |


## Region 13 failings

Region 13 is the Central Texas schools

In [19]:
region = campus.where(lambda row: row['REGION'] == '13')
print(len(region))

532


In [20]:
region_pivot = region.pivot('CHARTER', 'RATING')
region_pivot.print_table()

| CHARTER     | Met standard | Improvement required |
| ----------- | ------------ | -------------------- |
| Not charter |          475 |                   20 |
| Charter     |           35 |                    2 |


In [21]:
# filter to failed schools
region_failed = region.where(lambda row: row['RATING'] == 'Improvement required')

# sort the list by district, campus
region_sorted = region_failed.select(columns_fail_list).order_by(lambda row: (row['DISTNAME'], row['CAMPNAME']))

# print the list
region_sorted.print_table(max_rows=None, max_columns=None, max_column_width=25)

| CAMPNAME                  | DISTNAME                 | C_YRS_IR | CI1_MET | CI2_MET | CI3_MET | CI4_MET |
| ------------------------- | ------------------------ | -------- | ------- | ------- | ------- | ------- |
| BURNET M S                | AUSTIN ISD               |        2 | N       | N       | N       | Y       |
| GOVALLE EL                | AUSTIN ISD               |        1 | N       | Y       | N       | Y       |
| MARTIN MIDDLE             | AUSTIN ISD               |        1 | N       | N       | Y       | Y       |
| MENDEZ M S                | AUSTIN ISD               |        4 | N       | N       | N       | N       |
| BARTLETT SCHOOLS          | BARTLETT ISD             |        1 | N       | Y       | N       | N       |
| DIME BOX SCHOOL           | DIME BOX ISD             |        2 | N       | Y       | N       | Y       |
| BOOKER T WASHINGTON EL    | ELGIN ISD                |        1 | N       | Y       | N       | Y       |
| ELGIN EL                  